# **Team**

> Balestrieri Niccolò - 10936955 <br>
  Bertogalli Andrea - 10702303 <br>
  Cavalieri Francesco - 11020855    
  Tombini Nicolò - 10912627


# Non-stationary environment

In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px

from plotly.subplots import make_subplots

In [ ]:
np.random.seed(42)

In [ ]:
class Environment:
    def __init__(self):
        raise NotImplementedError

    def round(self, a_t):
        raise NotImplementedError

#Pricing

## Environments definition

In [ ]:
class NonStationaryPricingEnvironment(Environment):
    def __init__(self, time_steps, cost, prices,  sigma=0.3, change_treshold = 20):
        self.time_steps = time_steps
        self.change_treshold = change_treshold
        self.sigma = sigma
        self.mu = 0
        self.current_step = 0
        self.cost = cost
        self.prices = prices
        self.white_noise = None
        self.sample_white_noise()

    def purchase_probabilities(self):
      probabilities = []
      for i, price in enumerate(self.prices):
        base_probability = 1 - price
        probabilities.append(base_probability + self.white_noise[i])

      return np.clip(probabilities, 0, 1)

    def profit_curves(self):
      probabilities = []
      for i, price in enumerate(self.prices):
        base_probability = 1 - price
        probabilities.append(base_probability + self.white_noise[i])

      return probabilities * (self.prices - self.cost)

    def get_buying_probability(self, price):
      base_probability = 1 - price
      idx = np.where(self.prices == price)[0]
      return np.clip(base_probability + self.white_noise[idx], 0, 1)

    def sample_white_noise(self):
      self.white_noise = np.random.normal(self.mu, self.sigma, len(self.prices))

    def round(self, p_t, n_t):
      d_t = np.random.binomial(n_t, self.get_buying_probability(p_t))
      r_t = (p_t - self.cost) * d_t

      probs = self.purchase_probabilities()
      possible_rewards = [(self.prices[i] - self.cost) * np.random.binomial(n_t, probs[i]) for i in range(len(self.prices))]
      l_t = max(possible_rewards) - r_t

      if not self.current_step % self.change_treshold:
        self.sample_white_noise()

      if self.current_step < self.time_steps:
          self.current_step += 1

      stats = {
          'd_t': d_t,
          'l_t': l_t,
          'possible_rewards': possible_rewards,
          'max_reward': max(possible_rewards),
      }

      return r_t, stats

    def is_terminated(self):
      return self.current_step >= self.time_steps

## Plots over time

In [ ]:
rounds = 100
prices = np.linspace(0, 1, 100)
adversarial_pricing_env = NonStationaryPricingEnvironment(time_steps=rounds, cost = 0.3, prices = prices, change_treshold=20)

allProbabilities = []
dmcurves = []
profits = []

while not adversarial_pricing_env.is_terminated():
  probabilities = adversarial_pricing_env.purchase_probabilities()
  profit = adversarial_pricing_env.profit_curves()
  r_t, stats = adversarial_pricing_env.round(0.1, 10)
  dmcurves.append(probabilities)
  profits.append(profit)

profits = np.array(profits)
dmcurves = np.array(dmcurves)

The Demand Curve Trend Over Time.

In [ ]:
x = prices
y = np.arange(dmcurves.shape[0])
X, Y = np.meshgrid(x, y)

print(dmcurves)

Z = dmcurves

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

fig.update_layout(
    title="3D Plot of Multiple Distributions - demand",
    scene=dict(
        xaxis_title='Price',
        yaxis_title='Days',
        zaxis_title='Probability'
    )
)

fig.show()

[[1.         0.9484197  1.         ... 0.0985186  0.01163505 0.        ]
 [1.         1.         0.90040093 ... 0.02327194 0.         0.13863104]
 [1.         1.         0.90040093 ... 0.02327194 0.         0.13863104]
 ...
 [1.         0.90209335 0.79434985 ... 0.         0.         0.        ]
 [1.         0.90209335 0.79434985 ... 0.         0.         0.        ]
 [1.         0.90209335 0.79434985 ... 0.         0.         0.        ]]


In [ ]:
fig = go.Figure()

for i in [0, 50, 90]:
    y = [dmcurves[j][i] for j in range(len(prices))]
    fig.add_trace(go.Scatter(x=list(range(100)), y=y, mode='lines', name=f'Price {round(prices[i], 2)}'))

fig.update_layout(
    title='User Buying Probability Over Time',
    title_x=0.5,
    xaxis_title='Time',
    yaxis_title='Buying Probability',
    xaxis=dict(tickmode='linear', tick0=0, dtick=5, range=[0, 100]),
    yaxis=dict(range=[0, 1]),
    legend=dict(title='Price'),
    font=dict(size=12, family="Arial"),
)

fig.show()

Profit Curve Trend Over Time

In [ ]:
x = prices
y = np.arange(profits.shape[0])
X, Y = np.meshgrid(x, y)

Z = profits

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

fig.update_layout(
    title="3D Plot of Multiple Distributions - profit",
    scene=dict(
        xaxis_title='Price',
        yaxis_title='Days',
        zaxis_title='Profits'
    )
)

fig.show()

In [ ]:
num_distributions = dmcurves.shape[0]
distribution_size = dmcurves.shape[1]
distributions = dmcurves

fig = go.Figure()

custom_colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'lime', 'yellow', 'brown']


for i in range(num_distributions):
    fig.add_trace(go.Scatter(
        x=prices,
        y=distributions[i, :],
        mode='lines',
        name=f'Distribution {i}',
        line=dict(color=custom_colors[i//20])
    ))

fig.update_layout(
    title="2D Plot of Multiple Distributions - multi realization",
    xaxis_title="Price",
    yaxis_title="Value (offset by Distribution Index)",
    showlegend=True
)

fig.show()

distributions_mean = np.mean(distributions, axis=0)
distributions_sd = np.std(distributions, axis=0)

lower_bound = distributions_mean - distributions_sd / np.sqrt(num_distributions//20)
upper_bound = distributions_mean + distributions_sd / np.sqrt(num_distributions//20)
t_values = np.arange(num_distributions)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.concatenate([t_values, t_values[::-1]]),
    y=np.concatenate([lower_bound, upper_bound[::-1]]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=True,
    name='Uncertainty'
))

fig.add_trace(go.Scatter(
    x=t_values,
    y=distributions_mean,
    mode='lines',
    line=dict(color='blue'),
    name='Average Demand'
))

fig.update_layout(
    title="2D Plot of Multiple Distributions - mean trend",
    xaxis_title="Price",
    yaxis_title="Value (offset by Distribution Index)",
    showlegend=True
)

fig.show()

In [ ]:
num_distributions = profits.shape[0]
distribution_size = profits.shape[1]
distributions = profits

fig = go.Figure()

custom_colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'lime', 'yellow', 'brown']

for i in range(num_distributions):
    fig.add_trace(go.Scatter(
        x=prices,
        y=distributions[i, :],
        mode='lines',
        name=f'Distribution {i}',
        line=dict(color=custom_colors[i//20])
    ))

fig.update_layout(
    title="2D Plot of Multiple Distributions - multi realization",
    xaxis_title="Price",
    yaxis_title="Profit (offset by Distribution Index)",
    showlegend=True
)

fig.show()

distributions_mean = np.mean(distributions, axis=0)
distributions_sd = np.std(distributions, axis=0)

lower_bound = distributions_mean - distributions_sd / np.sqrt(num_distributions//20)
upper_bound = distributions_mean + distributions_sd / np.sqrt(num_distributions//20)
t_values = prices
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.concatenate([t_values, t_values[::-1]]),
    y=np.concatenate([lower_bound, upper_bound[::-1]]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=True,
    name='Uncertainty'
))

fig.add_trace(go.Scatter(
    x=t_values,
    y=distributions_mean,
    mode='lines',
    line=dict(color='blue'),
    name='Average Demand'
))

fig.update_layout(
    title="2D Plot of Multiple Distributions - mean trend",
    xaxis_title="Price",
    yaxis_title="Profit (offset by Distribution Index)",
    showlegend=True
)

fig.show()

## AGENTS

In [ ]:
class SWUCBAgent:
    def __init__(self, K, T, W, range=1):
        self.K = K
        self.T = T
        self.W = W
        self.range = range
        self.a_t = None
        self.cache = np.repeat(np.nan, repeats=K*W).reshape(W, K)
        self.N_pulls = np.zeros(K)
        self.t = 0

    def pull_arm(self):
        if self.t < self.K:
            self.a_t = self.t
        else:
            n_pulls_last_w = self.W - np.isnan(self.cache).sum(axis=0)
            avg_last_w = np.nanmean(self.cache, axis=0)
            ucbs = avg_last_w + self.range*np.sqrt(2*np.log(self.W)/n_pulls_last_w)
            self.a_t = np.argmax(ucbs)
        return self.a_t

    def update(self, r_t):
        self.N_pulls[self.a_t] += 1
        self.cache = np.delete(self.cache, (0), axis=0)
        new_samples = np.repeat(np.nan, self.K)
        new_samples[self.a_t] = r_t
        self.cache = np.vstack((self.cache, new_samples))
        self.t += 1

In [ ]:
class CUSUMUCBAgent:
    def __init__(self, K, T, M, h, alpha=0.99, range=1):
        self.K = K
        self.T = T
        self.M = M
        self.h = h
        self.alpha=alpha
        self.range = range
        self.a_t = None
        self.reset_times = np.zeros(K)
        self.N_pulls = np.zeros(K)
        self.all_rewards = [[] for _ in np.arange(K)]
        self.counters = np.repeat(M, K)
        self.average_rewards = np.zeros(K)
        self.n_resets = np.zeros(K)
        self.n_t = 0
        self.t = 0

    def pull_arm(self):
        if (self.counters > 0).any():
            for a in np.arange(self.K):
                if self.counters[a] > 0:
                    self.counters[a] -= 1
                    self.a_t = a
                    break
        else:
            if np.random.random() <= 1-self.alpha:
                ucbs = self.average_rewards + self.range*np.sqrt(np.log(self.n_t)/self.N_pulls)
                self.a_t = np.argmax(ucbs)
            else:
                self.a_t = np.random.choice(np.arange(self.K))
        return self.a_t

    def update(self, r_t):
        self.N_pulls[self.a_t] += 1
        self.all_rewards[self.a_t].append(r_t)
        if self.counters[self.a_t] == 0:
            if self.change_detection():
                self.n_resets[self.a_t] +=1
                self.N_pulls[self.a_t] = 0
                self.average_rewards[self.a_t] = 0
                self.counters[self.a_t] = self.M
                self.all_rewards[self.a_t] = []
                self.reset_times[self.a_t] = self.t
            else:
                self.average_rewards[self.a_t] += (r_t - self.average_rewards[self.a_t])/self.N_pulls[self.a_t]
        self.n_t = sum(self.N_pulls)
        self.t += 1

    def change_detection(self):
        u_0 = np.mean(self.all_rewards[self.a_t][:self.M])
        sp, sm = (np.array(self.all_rewards[self.a_t][self.M:])- u_0, u_0 - np.array(self.all_rewards[self.a_t][self.M:]))
        gp, gm = 0, 0
        for sp_, sm_ in zip(sp, sm):
            gp, gm = max([0, gp + sp_]), max([0, gm + sm_])
            if max([gp, gm]) >= self.h:
                return True
        return False

## SETTINGS AND RUNNING

### SW AGENT

In [ ]:
def runsSWUCB(W, T):
    n_trials = 20
    K = 10
    prices = np.linspace(0, 1, K)
    cost = 0.2
    T_change = 20

    n_t = 1

    regret_per_trial = []
    profits_ci = []

    for seed in range(n_trials):
        np.random.seed(seed)
        env = NonStationaryPricingEnvironment(time_steps=T, cost=cost, prices=prices, sigma=0.1, change_treshold=T_change)
        ucb_agent = SWUCBAgent(K, T, W)

        agent_rewards = np.array([])
        max_profits = np.array([])

        for t in range(T):
          expected_clairvoyant_rewards = np.max(env.profit_curves())
          max_profits = np.append(max_profits, expected_clairvoyant_rewards)
          a_t = ucb_agent.pull_arm()
          r_t, stats = env.round(prices[a_t], n_t)
          ucb_agent.update(r_t)
          agent_rewards = np.append(agent_rewards, r_t)

        cumulative_regret = np.cumsum(expected_clairvoyant_rewards - agent_rewards)
        regret_per_trial.append(cumulative_regret)
        profits_ci.append(agent_rewards)

    regret_per_trial = np.array(regret_per_trial)
    average_regret = regret_per_trial.mean(axis=0)
    regret_sd = regret_per_trial.std(axis=0)

    fig1 = go.Figure()

    fig1.add_trace(go.Scatter(
        x=np.arange(T), y=average_regret,
        mode='lines',
        name='Average Regret'
    ))

    fig1.add_trace(go.Scatter(
        x=np.arange(T),
        y=average_regret + regret_sd / np.sqrt(n_trials),
        fill=None,
        mode='lines',
        line=dict(color='lightblue'),
        showlegend=False
    ))

    fig1.add_trace(go.Scatter(
        x=np.arange(T),
        y=average_regret - regret_sd / np.sqrt(n_trials),
        fill='tonexty',
        mode='lines',
        line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    for x_pos in range(250, T + 1, 250):
        fig1.add_vline(x=x_pos, line=dict(color='red', dash='dash', width=0.8), name='Time Window')

    fig1.update_layout(
        title=f'Regret SW-UCB W={W}',
        xaxis_title='Time (t)',
        yaxis_title='Average Regret',
        showlegend=True
    )

    fig1.show()

    profits_ci = np.array(profits_ci)
    profits_ci_mean = profits_ci.mean(axis=0)
    profits_ci_sd = profits_ci.std(axis=0)

    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(
        x=np.arange(T), y=profits_ci_mean,
        mode='lines',
        name='Average Profit'
    ))

    fig2.add_trace(go.Scatter(
        x=np.arange(T),
        y=profits_ci_mean + profits_ci_sd / np.sqrt(n_trials),
        fill=None,
        mode='lines',
        line=dict(color='lightblue'),
        showlegend=False
    ))

    fig2.add_trace(go.Scatter(
        x=np.arange(T),
        y=profits_ci_mean - profits_ci_sd / np.sqrt(n_trials),
        fill='tonexty',
        mode='lines',
        line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    for x_pos in range(250, T + 1, 250):
        fig2.add_vline(x=x_pos, line=dict(color='red', dash='dash', width=0.8), name='Time Window')

    fig2.update_layout(
        title=f'Profits SW-UCB W={W}',
        xaxis_title='Time (t)',
        yaxis_title='Average Profit',
        showlegend=True
    )

    fig2.show()

In [ ]:
T = 100
U_T = 5
runsSWUCB(T = T, W=int(2*np.sqrt(T*np.log(T)/U_T)))
U_T = 100
runsSWUCB(T = T, W=int(2*np.sqrt(T*np.log(T)/U_T)))


<ipython-input-11-62103c9d4af3>:26: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

<ipython-input-11-62103c9d4af3>:17: RuntimeWarning:

Mean of empty slice

<ipython-input-11-62103c9d4af3>:18: RuntimeWarning:

divide by zero encountered in divide





```
# Questo è formattato come codice
```

### CUSUM AGENT

In [ ]:
def runsCUSUM(h, alpha, m, T, K, U_T):
    n_trials = 20
    K = K
    T = T
    prices = np.linspace(0, 1, K)
    cost = 0.2
    T_change = 20

    n_t = 20
    print(prices)

    regret_per_trial = []
    profits_ci = []
    resets_per_trial = []

    for seed in range(n_trials):
        np.random.seed(seed)
        env = NonStationaryPricingEnvironment(time_steps=T, cost=cost, prices=prices, sigma=0.1, change_treshold=T_change)
        ucb_agent = CUSUMUCBAgent(K=K, T=T, h=h, alpha=alpha, M=m)

        agent_rewards = np.array([])
        max_profits = np.array([])

        for t in range(T):
          expected_clairvoyant_rewards = np.max(env.profit_curves())*n_t
          max_profits = np.append(max_profits, expected_clairvoyant_rewards)
          a_t = ucb_agent.pull_arm()
          r_t, stats = env.round(prices[a_t], n_t)
          ucb_agent.update(r_t)
          agent_rewards = np.append(agent_rewards, r_t)

        cumulative_regret = np.cumsum( (expected_clairvoyant_rewards - agent_rewards) /n_t)
        regret_per_trial.append(cumulative_regret)
        profits_ci.append(agent_rewards/n_t)
        resets_per_trial.append(ucb_agent.n_resets)

    regret_per_trial = np.array(regret_per_trial)
    resets_per_trial = np.vstack(resets_per_trial)
    average_regret = regret_per_trial.mean(axis=0)
    regret_sd = regret_per_trial.std(axis=0)

    fig_regret = go.Figure()
    fig_regret.add_trace(go.Scatter(
        x=np.arange(T), y=average_regret,
        mode='lines', name='Average Regret'
    ))

    fig_regret.add_trace(go.Scatter(
        x=np.arange(T),
        y=average_regret + regret_sd / np.sqrt(n_trials),
        mode='lines', fill='tonexty',  line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    fig_regret.add_trace(go.Scatter(
        x=np.arange(T),
        y=average_regret - regret_sd / np.sqrt(n_trials),
        mode='lines', fill='tonexty',  line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    fig_regret.update_layout(
        title=f'Regret CUSUM-UCB  setting=( U_T = {U_T} , H={h:.2f} , alpha= {alpha:.2f} , M={m} )',
        xaxis_title='$t$'
    )

    for x_pos in range(250, T + 1, 250):
        fig_regret.add_vline(x=x_pos, line=dict(color='red', dash='dash', width=0.8))

    fig_regret.show()

    profits_ci = np.array(profits_ci)
    profits_ci_mean = profits_ci.mean(axis=0)
    profits_ci_sd = profits_ci.std(axis=0)

    fig_profit = go.Figure()
    fig_profit.add_trace(go.Scatter(
        x=np.arange(T), y=profits_ci_mean,
        mode='lines', name='Average Profit'
    ))
    fig_profit.add_trace(go.Scatter(
        x=np.arange(T),
        y=profits_ci_mean - profits_ci_sd / np.sqrt(n_trials),
        mode='lines', fill='tonexty', line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    fig_profit.add_trace(go.Scatter(
        x=np.arange(T),
        y=profits_ci_mean + profits_ci_sd / np.sqrt(n_trials),
        mode='lines', fill='tonexty',  line=dict(color='lightblue'),
        name='Uncertainty'
    ))

    fig_profit.update_layout(
        title=f'Profits CUSUM-UCB setting=( U_T = {U_T} , H={h:.2f} , alpha= {alpha:.2f} , M={m} )',
        xaxis_title='$t$'
    )

    for x_pos in range(250, T + 1, 250):
        fig_profit.add_vline(x=x_pos, line=dict(color='red', dash='dash', width=0.8))

    fig_profit.show()


In [ ]:
T = 100
U_T = 5
h = 2*np.log(T/U_T)
alpha = np.sqrt(U_T*np.log(T/U_T)/T)
M = max(round(np.log(T/U_T)) , 1)
K = 10

runsCUSUM(h=h , alpha=alpha , m=M , T = T , K=K , U_T = U_T)

[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


<ipython-input-12-4a07fdb4ba98>:46: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [ ]:
T = 100
U_T = 30
h = 2*np.log(T/U_T)
alpha = np.sqrt(U_T*np.log(T/U_T)/T)
M = max(round(np.log(T/U_T)) , 1)
K = 10

runsCUSUM(h=h , alpha=alpha , m=M , T = T , K=K , U_T=U_T)

[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


<ipython-input-12-4a07fdb4ba98>:46: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



# BONUS POINT

## Agents

In [ ]:
class RBFGaussianProcess:
    def __init__(self, scale=1, reg=1e-2):
        self.scale = scale
        self.reg = reg
        self.k_xx_inv = None

    def rbf_kernel_incr_inv(self, B, C, D):
        temp = np.linalg.inv(D - C @ self.k_xx_inv @ B)
        block1 = self.k_xx_inv + self.k_xx_inv @ B @ temp @ C @ self.k_xx_inv
        block2 = - self.k_xx_inv @ B @ temp
        block3 = - temp @ C @ self.k_xx_inv
        block4 = temp
        res1 = np.concatenate((block1, block2), axis=1)
        res2 = np.concatenate((block3, block4), axis=1)
        res = np.concatenate((res1, res2), axis=0)
        return res

    def rbf_kernel(self, a, b):
        a_ = a.reshape(-1, 1)
        b_ = b.reshape(-1, 1)
        output = -1 * np.ones((a_.shape[0], b_.shape[0]))
        for i in range(a_.shape[0]):
            output[i, :] = np.power(a_[i] - b_, 2).ravel()
        return np.exp(-self.scale * output)

    def fit(self, x=np.array([]), y=np.array([])):
        x,y = np.array(x),np.array(y)
        if self.k_xx_inv is None:
            self.y = y.reshape(-1,1)
            self.x = x.reshape(-1,1)
            k_xx = self.rbf_kernel(self.x, self.x) + self.reg * np.eye(self.x.shape[0])
            self.k_xx_inv = np.linalg.inv(k_xx)
        else:
            B = self.rbf_kernel(self.x, x)
            self.x = np.vstack((self.x, x))
            self.y = np.vstack((self.y, y))
            self.k_xx_inv = self.rbf_kernel_incr_inv(B, B.T, np.array([1 + self.reg]))

        return self

    def predict(self, x_predict):
        k = self.rbf_kernel(x_predict, self.x)

        mu_hat = k @ self.k_xx_inv @ self.y
        sigma_hat = 1 - np.diag(k @ self.k_xx_inv @ k.T)

        return mu_hat.ravel(), sigma_hat.ravel()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern
from sklearn.gaussian_process.kernels import DotProduct

class GPUCBAgent:
    def __init__(self, T, discretization=30):
        self.T = T
        self.discretization = discretization
        self.arms_p1 = np.linspace(0, 1, discretization)
        self.arms_p2 = np.linspace(0, 1, discretization)
        kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e3))

        self.gp_p1 = GaussianProcessRegressor(kernel = kernel, alpha=1e-2, random_state=0)
        self.a_t = None
        self.action_hist = np.array([])
        self.reward_hist = np.array([])
        self.mu_t_p1 = np.zeros(discretization)
        self.sigma_t_p1 = np.zeros(discretization)
        self.gamma = lambda t: np.log(t+1)**2
        self.beta = lambda t: 1 + 0.5*np.sqrt(2 * (self.gamma(t) + 1 + np.log(T)))
        self.N_pulls = np.zeros(discretization)
        self.t = 0
        self.possible_arms = np.empty((0, 2))
        self.profit_p1 = np.empty((0,))

    def pull_arm(self):
        range_p1, range_p2 = np.meshgrid(self.arms_p1, self.arms_p2)
        possible_arms = np.column_stack((range_p1.ravel(), range_p2.ravel()))
        self.mu_t_p1, self.sigma_t_p1 = self.gp_p1.predict(possible_arms, return_std=True)
        ucbs_p1 = self.mu_t_p1 + self.beta(self.t) * self.sigma_t_p1
        self.a_t = np.argmax(ucbs_p1)

        return possible_arms[self.a_t]

    def update(self, arms, profit_p1, profit_p2):
        arms = np.array(arms)
        self.possible_arms = np.vstack((self.possible_arms, arms))
        self.profit_p1 = np.append(self.profit_p1,  (profit_p1+profit_p2))
        self.action_hist = np.append(self.action_hist, arms)
        self.reward_hist = np.append(self.reward_hist, (profit_p1+profit_p2))
        self.gp_p1 = self.gp_p1.fit(self.possible_arms, self.profit_p1)
        self.t += 1

## Environment

In [ ]:
import math

### PROFIT CURVE 1

In [ ]:
class PricingStochasticEnvironment2Products:
    def __init__(self, conversion_probability, cost, prices1, prices2):
        self.conversion_probability = conversion_probability
        self.cost = cost
        self.prices1 = prices1
        self.prices2 = prices2
        self.mu = 0
        self.sigma = 0.04
        self.white_noise_p1 = None
        self.sample_white_noise_p1()
        self.white_noise_p2 = None
        self.sample_white_noise_p2()
        self.probabilities_p1 = np.zeros((len(prices1), len(prices2)))
        self.probabilities_p2 = np.zeros((len(prices1), len(prices2)))

    def sample_white_noise_p1(self):
      self.white_noise_p1 = np.random.normal(self.mu, self.sigma, len(self.prices1)**2)

    def sample_white_noise_p2(self):
      self.white_noise_p2 = np.random.normal(self.mu, self.sigma, len(self.prices2)**2)

    def get_profit_curves_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p1 * (self.prices1 - self.cost))[i, j]

    def get_profit_curves_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p2 * (self.prices1 - self.cost))[i, j]

    def profit_curves_p1(self):
      return self.probabilities_p1 * (self.prices1 - self.cost)

    def profit_curves_p2(self):
      return self.probabilities_p2 * (self.prices2 - self.cost)

    def get_demand_curve_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p1[i, j]

    def get_demand_curve_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p2[i, j]

    def demand_curve_p1(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          min_val = 0.2
          max_val = 0.8
          Z_parabola_scaled = -0.8 * ((p1 - 0.4)**2 + (p2 - 0.6)**2) + 1.0
          Z_parabola_scaled = (Z_parabola_scaled - min_val)
          Z_parabola_scaled /= max_val
          Z_parabola_scaled = Z_parabola_scaled * (max_val - min_val) + min_val
          base_probability = Z_parabola_scaled + self.white_noise_p1[i*j]

          self.probabilities_p1[i, j] = base_probability

      return np.clip(self.probabilities_p1, 0, 1)

    def demand_curve_p2(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          min_val = 0.2
          max_val = 0.8
          Z_parabola_scaled = -0.8 * ((p1 - 0.2)**2 + (p2 - 0.3)**2) + 1.0
          Z_parabola_scaled = (Z_parabola_scaled - min_val)
          Z_parabola_scaled /= max_val
          Z_parabola_scaled = Z_parabola_scaled * (max_val - min_val) + min_val
          base_probability = Z_parabola_scaled + self.white_noise_p2[i*j]
          self.probabilities_p2[i, j] = base_probability

      return np.clip(self.probabilities_p2, 0, 1)

    def round(self, p_t, n_t):
        d_t = np.random.binomial(n_t, self.conversion_probability(p_t))
        r_t = (p_t - self.cost)*d_t
        return d_t, r_t

### PROFIT CURVE 2

In [ ]:
class PricingStochasticEnvironment2Products:
    def __init__(self, conversion_probability, cost, prices1, prices2):
        self.conversion_probability = conversion_probability
        self.cost = cost
        self.prices1 = prices1
        self.prices2 = prices2
        self.mu = 0
        self.sigma = 0.1
        self.white_noise_p1 = None
        self.sample_white_noise_p1()
        self.white_noise_p2 = None
        self.sample_white_noise_p2()
        self.probabilities_p1 = np.zeros((len(prices1), len(prices2)))
        self.probabilities_p2 = np.zeros((len(prices1), len(prices2)))

    def sample_white_noise_p1(self):
      self.white_noise_p1 = np.random.normal(self.mu, self.sigma, len(self.prices1)**2)

    def sample_white_noise_p2(self):
      self.white_noise_p2 = np.random.normal(self.mu, self.sigma, len(self.prices2)**2)

    def get_profit_curves_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p1 * (self.prices1 - self.cost))[i, j]

    def get_profit_curves_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p2 * (self.prices2 - self.cost))[i, j]

    def profit_curves_p1(self):
      return self.probabilities_p1 * (self.prices1 - self.cost)

    def profit_curves_p2(self):
      return self.probabilities_p2 * (self.prices2 - self.cost)

    def get_demand_curve_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p1[i, j]

    def get_demand_curve_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p2[i, j]

    def demand_curve_p1(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          base_probability = -0.4 * p2 + 0.6 * p1 + 0.4 + 0.3 * math.sin(4 * p1) + self.white_noise_p1[i*j]

          self.probabilities_p1[i, j] = base_probability

      return np.clip(self.probabilities_p1, 0, 1)

    def demand_curve_p2(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          base_probability = 0.4 * p2 - 0.6 * p1 + 0.4 + self.white_noise_p2[i*j] + 0.3 * math.sin(4 * p2)
          self.probabilities_p2[i, j] = base_probability

      return np.clip(self.probabilities_p2, 0, 1)

    def round(self, p_t, n_t):
        d_t = np.random.binomial(n_t, self.conversion_probability(p_t))
        r_t = (p_t - self.cost)*d_t
        return d_t, r_t

### PROFIT CURVE 3

In [ ]:
class PricingStochasticEnvironment2Products:
    def __init__(self, conversion_probability, cost, prices1, prices2):
        self.conversion_probability = conversion_probability
        self.cost = cost
        self.prices1 = prices1
        self.prices2 = prices2
        self.mu = 0
        self.sigma = 0.02
        self.white_noise_p1 = None
        self.sample_white_noise_p1()
        self.white_noise_p2 = None
        self.sample_white_noise_p2()
        self.probabilities_p1 = np.zeros((len(prices1), len(prices2)))
        self.probabilities_p2 = np.zeros((len(prices1), len(prices2)))

    def sample_white_noise_p1(self):
      self.white_noise_p1 = np.random.normal(self.mu, self.sigma, len(self.prices1)**2)

    def sample_white_noise_p2(self):
      self.white_noise_p2 = np.random.normal(self.mu, self.sigma, len(self.prices2)**2)

    def get_profit_curves_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p1 * (self.prices1 - self.cost))[i, j]

    def get_profit_curves_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p2 * (self.prices2 - self.cost))[i, j]

    def profit_curves_p1(self):
      return self.probabilities_p1 * (self.prices1 - self.cost)

    def profit_curves_p2(self):
      return self.probabilities_p2 * (self.prices2 - self.cost)

    def get_demand_curve_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p1[i, j]

    def get_demand_curve_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p2[i, j]

    def demand_curve_p1(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          base_probability = -0.2 * p2 + 0.6 * p1 + 0.4 + self.white_noise_p1[i*j]

          self.probabilities_p1[i, j] = base_probability

      return self.probabilities_p1
      #return np.clip(probabilities, 0, 1)

    def demand_curve_p2(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          base_probability = 0.2 * p2 - 0.6 * p1 + 0.4 + self.white_noise_p1[i*j]
          self.probabilities_p2[i, j] = base_probability

      return self.probabilities_p2

    def round(self, p_t, n_t):
        d_t = np.random.binomial(n_t, self.conversion_probability(p_t))
        r_t = (p_t - self.cost)*d_t
        return d_t, r_t

### PROFIT CURVE 4

In [ ]:
class PricingStochasticEnvironment2Products:
    def __init__(self, conversion_probability, cost, prices1, prices2):
        self.conversion_probability = conversion_probability
        self.cost = cost
        self.prices1 = prices1
        self.prices2 = prices2
        self.mu = 0
        self.sigma = 0.02
        self.white_noise_p1 = None
        self.sample_white_noise_p1()
        self.white_noise_p2 = None
        self.sample_white_noise_p2()
        self.probabilities_p1 = np.zeros((len(prices1), len(prices2)))
        self.probabilities_p2 = np.zeros((len(prices1), len(prices2)))

    def sample_white_noise_p1(self):
      self.white_noise_p1 = np.random.normal(self.mu, self.sigma, len(self.prices1)**2)

    def sample_white_noise_p2(self):
      self.white_noise_p2 = np.random.normal(self.mu, self.sigma, len(self.prices2)**2)

    def get_profit_curves_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p1 * (self.prices1 - self.cost))[i, j]

    def get_profit_curves_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return (self.probabilities_p2 * (self.prices2 - self.cost))[i, j]

    def profit_curves_p1(self):
      return self.probabilities_p1 * (self.prices1 - self.cost)

    def profit_curves_p2(self):
      return self.probabilities_p2 * (self.prices2 - self.cost)

    def get_demand_curve_p1(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p1[i, j]

    def get_demand_curve_p2(self, p1, p2):
      i = np.where(self.prices1 == p1)[0][0]
      j = np.where(self.prices2 == p2)[0][0]
      return self.probabilities_p2[i, j]

    def demand_curve_p1(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          base_probability = +0.05 * p2 - 0.6 * p1**2 + 0.4 + self.white_noise_p1[i*j]

          self.probabilities_p1[i, j] = base_probability

      return self.probabilities_p1

    def demand_curve_p2(self):
      probabilities = np.zeros((len(p1_range), len(p2_range)))

      for i, p1 in enumerate(self.prices1):
        for j, p2 in enumerate(self.prices2):
          bell1 = -2 * np.exp(-((p1 - 0.4) ** 2 / (2 * .1 ** 2) + (p2 - .4) ** 2 / (2 * .1 ** 2)))
          bell2 = 6 * np.exp(-( ( (p1 - .7)/2) ** 2 / (2 * .1 ** 2) + (p2 - .7) ** 2 / (2 * .1 ** 2)))
          base_probability = 1-  p2 + 0.6 * p1*p2 + 0.4 + self.white_noise_p1[i*j] + bell1 + bell2
          self.probabilities_p2[i, j] = base_probability
      self.probabilities_p2 = (self.probabilities_p2 - self.probabilities_p2.min()) / (self.probabilities_p2.max() - self.probabilities_p2.min())
      return self.probabilities_p2

    def round(self, p_t, n_t):
        d_t = np.random.binomial(n_t, self.conversion_probability(p_t))
        r_t = (p_t - self.cost)*d_t
        return d_t, r_t

## Create Envitroment

In [ ]:
conversion_probability = lambda p: 1-p
p1_range = np.linspace(0, 1, 20)
p2_range = np.linspace(0, 1, 20)
adversarial_pricing_env = PricingStochasticEnvironment2Products(conversion_probability=conversion_probability, cost = 0.3, prices1 = p1_range, prices2 = p2_range)
D1 = adversarial_pricing_env.demand_curve_p1()
D2 = adversarial_pricing_env.demand_curve_p2()

## Demand curves plot

In [ ]:
def plot_demand_curve():
    D1 = adversarial_pricing_env.demand_curve_p1()
    D2 = adversarial_pricing_env.demand_curve_p2()

    P1, P2 = np.meshgrid(p1_range, p2_range)

    fig = go.Figure(data=[
        go.Surface(z=D1, x=P1, y=P2, colorscale='Viridis', showscale=True)
    ])
    fig.update_layout(
        title='Demand Curve for the first product',
        scene=dict(
            xaxis_title='First product price',
            yaxis_title='Second product price',
            zaxis_title='Demand'
        )
    )
    fig.show()

    fig2 = go.Figure(data=[
        go.Surface(z=D2, x=P1, y=P2, colorscale='Cividis', showscale=True)
    ])
    fig2.update_layout(
        title='Demand Curve for the second product',
        scene=dict(
            xaxis_title='First product price',
            yaxis_title='Second product price',
            zaxis_title='Demand'
        )
    )
    fig2.show()

plot_demand_curve()

In [ ]:
def plot_profit_curve():
    D1 = adversarial_pricing_env.profit_curves_p1()
    D2 = adversarial_pricing_env.profit_curves_p2()
    P1, P2 = np.meshgrid(p1_range, p2_range)

    max_profit_p1 = np.max(D1)
    best_indices_1 = np.where(D1>= max_profit_p1)
    max_profit_p2 = np.max(D2)
    best_indices_2 = np.where(D2>= max_profit_p2)

    fig = go.Figure(data=[
        go.Surface(z=D1, x=P1, y=P2, colorscale='Viridis', showscale=True)
    ])
    fig.update_layout(
        title='Profit Curve for the first product',
        scene=dict(
            xaxis_title='First  price',
            yaxis_title='Second  price',
            zaxis_title='Profit'
        )
    )

    fig.add_trace(
    go.Scatter3d(
        x=p1_range[best_indices_1[1]],
        y=p2_range[best_indices_1[0]],
        z=np.repeat(max_profit_p1 ,best_indices_1[0].shape[0]),
        mode='markers',
        marker=dict(
            size=2,
            color='red',
            symbol='circle'
        ),
        name='Max Profit Point'
    )
    )

    fig.show()

    fig2 = go.Figure(data=[
        go.Surface(z=D2, x=P1, y=P2, colorscale='Cividis', showscale=True)
    ])


    fig2.add_trace(
        go.Scatter3d(
            x=p1_range[best_indices_2[1]],
            y=p2_range[best_indices_2[0]],
            z=np.repeat(max_profit_p2 ,best_indices_2[0].shape[0]),
            mode='markers',
            marker=dict(
                size=2,
                color='red',
                symbol='circle'
            ),
            name='Max Profit Point'
        )
    )


    fig2.update_layout(
        title='Profit Curve for the second product',
        scene=dict(
            xaxis_title='First  price',
            yaxis_title='Second  price',
            zaxis_title='Profit'
        )
    )

    fig2.show()


    D3 = D1 + D2

    fig3 = go.Figure(data=[
        go.Surface(z=D3, x=P1, y=P2, colorscale='Cividis', showscale=True)
    ])


    fig3.update_layout(
        title='Profit Curve Combined',
        scene=dict(
            xaxis_title='First  price',
            yaxis_title='Second  price',
            zaxis_title='Profit'
        )
    )

    fig3.show()

plot_profit_curve()

In [ ]:
def plot_profit_heatMap():

    x_labels = ["0" , "1"]
    y_labels = ["0" , "1"]
    D1 = adversarial_pricing_env.profit_curves_p1()

    D2 = adversarial_pricing_env.profit_curves_p2()
    D3 = D1 + D2

    best_indices = np.where(D3>= np.max(D3))

    fig = px.imshow(D1, color_continuous_scale='viridis', labels=dict(color="Value"))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=[0,p1_range.shape[0]-1],
            ticktext=x_labels
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=[p2_range.shape[0]-1,0],
            ticktext=y_labels
        ),
        title="Heatmap P1",
        xaxis_title="Custom X-axis",
        yaxis_title="Custom Y-axis"
    )

    fig.show()

    fig = px.imshow(D2, color_continuous_scale='viridis', labels=dict(color="Value"))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=[0,p1_range.shape[0]-1],
            ticktext=x_labels
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=[p2_range.shape[0]-1,0],
            ticktext=y_labels
        ),
        title="Heatmap P2",
        xaxis_title="Custom X-axis",
        yaxis_title="Custom Y-axis"
    )

    fig.show()

    fig = px.imshow(D3, color_continuous_scale='viridis', labels=dict(color="Value"))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=[0,p1_range.shape[0]-1],
            ticktext=x_labels
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=[p2_range.shape[0]-1,0],
            ticktext=y_labels
        ),
        title="Heatmap Combined",
        xaxis_title="Custom X-axis",
        yaxis_title="Custom Y-axis"
    )

    fig.show()


plot_profit_heatMap()

## Clairvoyant calculation


## Interaction

In [ ]:
n_trials = 1
regrets = []
agents = []
rewards_2D = []
for trial in range(n_trials):

    T = 300
    ################### clairvoyant part ####################################
    adversarial_pricing_env = PricingStochasticEnvironment2Products(conversion_probability=conversion_probability, cost = 0.3, prices1 = p1_range, prices2 = p2_range)
    D1 = adversarial_pricing_env.demand_curve_p1()
    D2 = adversarial_pricing_env.demand_curve_p2()

    profit_matrix_p1 = adversarial_pricing_env.profit_curves_p1()
    profit_matrix_p2 = adversarial_pricing_env.profit_curves_p2()

    profit_matrix = profit_matrix_p1 + profit_matrix_p2
    max_profit = np.max(profit_matrix)

    best_indices = np.where(profit_matrix>= max_profit)
    clairvoyant_expected_rewards = np.full(T, max_profit)
    ### Agent / iteration part ################################################
    agent = GPUCBAgent(T=T, discretization=20)
    agent_rewards = np.array([])

    for t in range(T):

        p1, p2 = agent.pull_arm()
        d1 = adversarial_pricing_env.get_demand_curve_p1(p1, p2)
        d2 = adversarial_pricing_env.get_demand_curve_p2(p1, p2)
        profit_p1= adversarial_pricing_env.get_profit_curves_p1(p1, p2)
        profit_p2 = adversarial_pricing_env.get_profit_curves_p2(p1, p2)
        print(f"Time = { (t ,(p1, p2) ,  (profit_p1, profit_p2) , (profit_p1 + profit_p2)) }")
        agent.update([p1, p2], profit_p1, profit_p2)
        agent_rewards = np.append(agent_rewards, profit_p1 + profit_p2)

    expected_clairvoyant_rewards = np.repeat(max_profit, T)
    cumulative_regret = np.cumsum(expected_clairvoyant_rewards - agent_rewards)
    agents.append(agent)
    regrets.append(cumulative_regret)
    rewards_2D.append(agent_rewards)
regrets = np.array(regrets)
regret_avg = np.mean(regrets, axis=0)
regret_std = np.std(regrets, axis=0)
rewards_2D = np.array(rewards_2D)

Time = (0, (0.0, 0.0), (-0.12664015395092101, -0.203201225112325), -0.329841379063246)
Time = (1, (0.894736842105263, 0.0), (-0.1113825362223892, -0.12352255475221421), -0.2349050909746034)
Time = (2, (0.47368421052631576, 0.7368421052631579), (0.3143393307899951, 0.2627059826059936), 0.5770453133959887)
Time = (3, (0.3684210526315789, 0.7894736842105263), (0.4021361038867805, 0.331143756262699), 0.7332798601494794)
Time = (4, (0.0, 1.0), (0.3794936925521491, 0.3061361919287584), 0.6856298844809074)
Time = (5, (1.0, 1.0), (0.3072156668952363, 0.08019215920431902), 0.38740782609955526)
Time = (6, (0.42105263157894735, 1.0), (0.5445932198947803, 0.30571407473915946), 0.8503072946339397)
Time = (7, (0.3684210526315789, 1.0), (0.49454379535467774, 0.294832001407853), 0.7893757967625308)
Time = (8, (0.3684210526315789, 1.0), (0.49454379535467774, 0.294832001407853), 0.7893757967625308)
Time = (9, (0.0, 1.0), (0.3794936925521491, 0.3061361919287584), 0.6856298844809074)
Time = (10, (0.421052

## PLOT RESULTS

### PLOT ESTIMATED HEATMAP

In [ ]:
grid_x, grid_y = np.meshgrid(p1_range, p2_range)
result = np.column_stack(( grid_y.ravel() , grid_x.ravel()))
predicted_heatap = agents[0].gp_p1.sample_y(result)

In [ ]:
x_labels = ["0" , "1"]
y_labels = ["0" , "1"]
heatmap_data = predicted_heatap.reshape(20, 20)
fig = px.imshow(heatmap_data, color_continuous_scale='viridis', labels=dict(color="Value"))

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=[0,p1_range.shape[0]-1],
        ticktext=x_labels
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=[p2_range.shape[0]-1,0],
        ticktext=y_labels
    ),
    title="Heatmap Estimated",
    xaxis_title="Price 1",
    yaxis_title="Price 2"
)

fig.show()

x_labels = ["0" , "1"]
y_labels = ["0" , "1"]
D1 = adversarial_pricing_env.profit_curves_p1()

D2 = adversarial_pricing_env.profit_curves_p2()
D3 = D1 + D2

fig = px.imshow(D3, color_continuous_scale='viridis', labels=dict(color="Value"))

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=[0,p1_range.shape[0]-1],
        ticktext=y_labels
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=[p2_range.shape[0]-1,0],
        ticktext=x_labels
    ),
    title="Heatmap Combined",
    xaxis_title="Custom X-axis",
    yaxis_title="Custom Y-axis"
)

fig.show()

### PLOT ESTIMATED CURVE 3D

In [ ]:
P1, P2 = np.meshgrid(p1_range, p2_range)

fig = go.Figure()

fig.add_trace(
    go.Surface(
        z=profit_matrix,
        x=P1,
        y=P2,
        colorscale='Viridis',
        opacity=0.8
    )
)

fig.add_trace(
    go.Surface(
        z=heatmap_data,
        x=P1,
        y=P2,
        colorscale='magma',
        opacity=0.5
    )
)

fig.update_layout(
    scene=dict(
        xaxis_title="P1",
        yaxis_title="P2",
        zaxis_title="Profit",
    ),
    title="Estimated Profit"
)

fig.show()

### PLOT REGRET

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(T),
    y=regret_avg,
    mode='lines',
    name='Mean Cumulative Regret'
))
fig.add_trace(go.Scatter(
    x=np.arange(T),
    y=regret_avg + regret_std / np.sqrt(n_trials),
    mode='lines',
    line=dict(width=0),
    name='Uncertainity',
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=np.arange(T),
    y=regret_avg - regret_std / np.sqrt(n_trials),
    fill='tonexty',
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    name='Uncertainity',
    showlegend=True
))
fig.update_layout(
    title='Mean GPUCB Cumulative Regret with uncertainity',
    xaxis_title='Round',
    yaxis_title='Mean Cumulative Regret'
)
fig.show()